## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Paamiut,GL,61.9940,-49.6678,36.50,83,12,6.58,few clouds
1,1,Tasiilaq,GL,65.6145,-37.6368,41.16,98,100,8.01,overcast clouds
2,2,Rawson,AR,-43.3002,-65.1023,46.87,44,50,26.60,scattered clouds
3,3,Jamestown,US,42.0970,-79.2353,72.95,63,26,4.00,scattered clouds
4,4,Busselton,AU,-33.6500,115.3333,54.79,73,100,4.25,overcast clouds
5,5,Walvis Bay,NaN,-22.9575,14.5053,68.95,56,0,16.11,clear sky
6,6,Maragogi,BR,-9.0122,-35.2225,80.15,82,77,9.15,broken clouds
7,7,Abu Dhabi,AE,24.4667,54.3667,93.31,50,0,6.91,clear sky
8,8,Juruti,BR,-2.1522,-56.0922,84.45,70,90,4.14,overcast clouds
9,9,Dingle,PH,10.9995,122.6711,77.79,86,73,2.82,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
hotel_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
hotel_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Jamestown,US,42.0970,-79.2353,72.95,63,26,4.00,scattered clouds
9,9,Dingle,PH,10.9995,122.6711,77.79,86,73,2.82,broken clouds
14,14,Sehithwa,BW,-20.4667,22.7167,71.19,24,0,0.38,clear sky
16,16,Muros,ES,42.7762,-9.0603,73.17,72,53,6.38,broken clouds
20,20,Quatre Cocos,MU,-20.2078,57.7625,73.51,57,14,20.00,few clouds
22,22,Sikeston,US,36.8767,-89.5879,78.73,57,0,11.50,clear sky
27,27,Big Rapids,US,43.6981,-85.4837,73.47,63,0,10.36,clear sky
48,48,Soyo,AO,-6.1349,12.3689,76.86,77,47,10.98,scattered clouds
56,56,Guaratinga,BR,-16.5667,-39.5667,77.45,74,90,4.79,overcast clouds
58,58,Ponta Do Sol,PT,32.6667,-17.1000,74.03,74,98,11.10,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
hotel_df.count()

City_ID                136
City                   136
Country                133
Lat                    136
Lng                    136
Max Temp               136
Humidity               136
Cloudiness             136
Wind Speed             136
Current Description    136
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_hotel_df = hotel_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
clean_hotel_df = clean_hotel_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
clean_hotel_df["Hotel Name"] = ""
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Jamestown,US,72.95,scattered clouds,42.0970,-79.2353,
9,Dingle,PH,77.79,broken clouds,10.9995,122.6711,
14,Sehithwa,BW,71.19,clear sky,-20.4667,22.7167,
16,Muros,ES,73.17,broken clouds,42.7762,-9.0603,
20,Quatre Cocos,MU,73.51,few clouds,-20.2078,57.7625,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in clean_hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        clean_hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping.")

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = clean_hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Jamestown,US,72.95,scattered clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
9,Dingle,PH,77.79,broken clouds,10.9995,122.6711,SEGAYA PROPERTY
14,Sehithwa,BW,71.19,clear sky,-20.4667,22.7167,MONLEK GUEST HOUSE
16,Muros,ES,73.17,broken clouds,42.7762,-9.0603,Apartment Av. Castelao
20,Quatre Cocos,MU,73.51,few clouds,-20.2078,57.7625,Sunrise Attitude Hotel
22,Sikeston,US,78.73,clear sky,36.8767,-89.5879,Drury Inn & Suites Sikeston
27,Big Rapids,US,73.47,clear sky,43.6981,-85.4837,"Holiday Inn Big Rapids, an IHG Hotel"
48,Soyo,AO,76.86,scattered clouds,-6.1349,12.3689,Hotel Mpampa
56,Guaratinga,BR,77.45,overcast clouds,-16.5667,-39.5667,Hotel Caraíva
58,Ponta Do Sol,PT,74.03,overcast clouds,32.6667,-17.1000,Hotel do Campo


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weahter Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))